In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-07-05 12:52:57.200283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:52:57.232283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-05 12:52:57.232443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import tensorflow as tf
tf.keras.utils.get_file('horse-or-human.zip',"https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip")

'/home/piyush/.keras/datasets/horse-or-human.zip'

In [3]:
import os
import zipfile #to handle zip file

local_zip = '/home/piyush/.keras/datasets/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('horse-or-human')
zip_ref.close()

In [4]:
os.listdir('horse-or-human')

['humans', 'horses']

In [5]:
os.listdir('horse-or-human/validation')

FileNotFoundError: [Errno 2] No such file or directory: 'horse-or-human/validation'

In [ ]:
os.listdir('horse-or-human/validation/horses')

In [ ]:
len(os.listdir('horse-or-human/validation/horses'))

In [ ]:
len(os.listdir('horse-or-human/validation/humans'))

In [ ]:
len(os.listdir('horse-or-human/train/horses'))

In [ ]:
len(os.listdir('horse-or-human/train/humans'))

In [ ]:
for fname in os.listdir('horse-or-human/train/humans'):
  print(fname)

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [ ]:
image_1 = load_img('horse-or-human/train/humans/human09-10.png')

In [ ]:
image_1

In [ ]:
image_2 = load_img('horse-or-human/validation/horses/horse39-3.png')

In [ ]:
image_2

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = os.path.join('horse-or-human')

In [ ]:
path

In [ ]:
train_dir = os.path.join(path,'train')

In [ ]:
train_dir

In [ ]:
validation_dir = os.path.join(path,'validation')

In [ ]:
validation_dir

In [ ]:
train_horses_dir = os.path.join(train_dir,'horses')
train_humans_dir = os.path.join(train_dir,'humans')

In [ ]:
train_horses_dir

In [ ]:
train_humans_dir

In [ ]:
validation_horses_dir = os.path.join(validation_dir,'horses')
validation_humans_dir = os.path.join(validation_dir,'humans')

In [ ]:
validation_horses_dir

In [ ]:
validation_humans_dir

In [ ]:
os.listdir(validation_humans_dir)

In [ ]:
type(os.listdir(validation_humans_dir))

In [ ]:
num_horses_tr = len(os.listdir(train_horses_dir))
num_humans_tr = len(os.listdir(train_humans_dir))
num_horses_val = len(os.listdir(validation_horses_dir))
num_humans_val = len(os.listdir(validation_humans_dir))
print('Horses training data ',num_horses_tr)
print('Humans training Data ',num_humans_tr)
print('Horses Valid data ',num_horses_val)
print('Humans valid data ',num_humans_val)

In [ ]:
total_train=num_horses_tr+num_humans_tr
total_val=num_horses_val+num_humans_val
print('Total Training images',total_train)
print('Total validation images',total_val)

In [ ]:
batch_size = 100
epochs = 20
IMG_HIEGHT = 300
IMG_WIDHT = 300

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                           class_mode='binary')

In [ ]:
train_data_gen.class_indices

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                              class_mode='binary')

In [ ]:
sample_train_images, label = next(train_data_gen)

In [ ]:
label

In [ ]:
sample_train_images.shape

In [ ]:
sample_train_images[10]

In [ ]:
plt.imshow(sample_train_images[10])

In [ ]:
sample_train_images[10].shape

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,5,figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_train_images[0:5])

In [ ]:
model = Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_HIEGHT,IMG_WIDHT,3)),
    MaxPooling2D(),
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history = model.fit(train_data_gen,steps_per_epoch=total_train//batch_size,epochs=epochs,
                             validation_data=val_data_gen,validation_steps=total_val//batch_size)

In [ ]:
history.history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Val LOss')
plt.legend(loc=0)
plt.title('Loss')
plt.show()

In [ ]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                    rotation_range=45,
                                    width_shift_range=.15,
                                    height_shift_range=.15,
                                    horizontal_flip=True,
                                    zoom_range=0.5)

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                    directory=train_dir,
                                                    shuffle=True,
                                                    target_size=(IMG_HIEGHT,IMG_HIEGHT),
                                                    class_mode='binary')

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                directory=validation_dir,
                                                target_size=(IMG_HIEGHT,IMG_WIDHT),
                                                class_mode='binary')

In [ ]:
model_new = Sequential([
    Conv2D(16,3,padding='same',activation='relu',input_shape=(IMG_HIEGHT,IMG_WIDHT,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64,3,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

In [ ]:
model_new.summary()

In [ ]:
epochs = 100

In [ ]:
model_new.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model_new.fit(train_data_gen,steps_per_epoch=total_train//batch_size,
                                 epochs=epochs,
                                 validation_data=val_data_gen,
                                 validation_steps=total_val//batch_size)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss,label='Val LOss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.models.save_model(model_new,'horses_humans.h5')

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('horses_humans.h5')

In [ ]:
model.summary()

In [ ]:
img = load_img('human01-23.png',target_size=(300,300))

In [ ]:
img

In [ ]:
img = img_to_array(img)

In [ ]:
img

In [ ]:
img.shape

In [ ]:
img = img.reshape(1,300,300,3)

In [ ]:
img.shape

In [ ]:
model.predict(img)

In [ ]:
img2 = load_img('horse03-5.png',target_size=(300,300))

In [ ]:
img2

In [ ]:
img2.shape

In [ ]:
img2 = img.reshape(1,300,300,3)

In [ ]:
img2.shape

In [ ]:
model.predict(img2)